In [1]:
from multiprocessing import Pool
import numpy as np
import random
from itertools import combinations as cb
import math
from copy import deepcopy as dc
from tqdm import tqdm

In [2]:
"""Evaluate Function """
class Evaluate:
    def __init__(self):
        None
    def evaluate(self,gen):
        None
    def check_dimentions(self,dim):
        None

"""Common Function"""
def random_search(n,dim):
    """
    create genes list
    input:{ n: Number of population, default=20
            dim: Number of dimension
    }
    output:{genes_list → [[0,0,0,1,1,0,1,...]...n]
    }
    """
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
    return gens

In [3]:
"""BBA"""
def BBA(Eval_Func,n=20,m_i=200,dim=None,minf=0,prog=False,qmin=0,qmax=2,loud_A=0.25,r=0.4,mp=None):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            m_i: Number of max iteration, default=300
            minf: minimazation flag, default=0, 0=maximization, 1=minimazation
            dim: Number of feature, default=None
            prog: Do you want to use a progress bar?, default=False
            qmin: frequency minimum to step
            qmax: frequency maximum to step
            loud_A: value of Loudness, default=0.25
            r: Pulse rate, default=0.4, Probability to relocate near the best position
            }
    output:{Best value: type float 0.967
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    estimate=Eval_Func().evaluate
    if dim==None:
        dim=Eval_Func().check_dimentions(dim)
    #flag=dr
    #qmin=0
    #qmax=2
    #loud_A=0.25
    #r=0.1
    #n_iter=0
    gens_dict={tuple([0]*dim):float("-inf") if minf == 0 else float("inf")}
    q=[0 for i in range(n)]
    v=[[0 for d in range(dim)] for i in range(n)]
    #cgc=[0 for i in range(max_iter)]
    fit=[float("-inf") if minf == 0 else float("inf") for i in range(n)]
    #dr=False
    gens=random_search(n,dim)#[[random.choice([0,1]) for d in range(dim)] for i in range(n)]

    if mp!=None:
        with Pool(mp) as p:
            alter_fit = p.map(estimate,gens)
        # pbest=dc(fit)
        # xpbest=dc(gens)
        for i,f in enumerate(alter_fit):
            fit[i]=f
            gens_dict[tuple(gens[i])]=f

    else:
        for i in range(n):
            if  tuple(gens[i]) in gens_dict:
                fit[i]=gens_dict[tuple(gens[i])]
            else:
                fit[i]=estimate(gens[i])
                gens_dict[tuple(gens[i])]=fit[i]

    if minf==0:
        maxf=max(fit)
        best_v=maxf
        best_s=gens[fit.index(max(fit))]
    elif minf==1:
        minf=min(fit)
        best_v=minf
        best_s=gens[fit.index(min(fit))]


    if prog:
        miter=tqdm(range(m_i))
    else:
        miter=range(m_i)

    if mp ==None:
        for it in miter:
            #cgc[i]=maxf
            for i in range(n):
                for j in range(dim):
                    q[i]=qmin+(qmin-qmax)*random.random()
                    v[i][j]=v[i][j]+(gens[i][j]-best_s[j])*q[i]

                    vstf=abs((2/math.pi)*math.atan((math.pi/2)*v[i][j]))

                    if random.random()<vstf:
                        gens[i][j]= 0 if gens[i][j]==1 else 1
                    else:
                        pass

                    if random.random()>r:
                        gens[i][j]=best_s[j]

                if  tuple(gens[i]) in gens_dict:
                    fnew=gens_dict[tuple(gens[i])]
                else:
                    fnew=estimate(gens[i])
                    gens_dict[tuple(gens[i])]=fnew

                if fnew >= fit[i] and random.random() < loud_A if minf==0 else fnew <= fit[i] and random.random() < loud_A:#max?
                    gens[i]=gens[i]
                    fit[i]=fnew

                if fnew>best_v if minf==0 else fnew<best_v:
                    best_s=dc(gens[i])
                    best_v=dc(fnew)
                    #ここから並列化処理書いてる
    else:
        for it in miter:
            for i in range(n):
                for j in range(dim):
                    q[i]=qmin+(qmin-qmax)*random.random()
                    v[i][j]=v[i][j]+(gens[i][j]-best_s[j])*q[i]

                    vstf=abs((2/math.pi)*math.atan((math.pi/2)*v[i][j]))

                    if random.random()<vstf:
                        gens[i][j]= 0 if gens[i][j]==1 else 1
                    else:
                        pass

                    if random.random()>r:
                        gens[i][j]=best_s[j]

            fnew=[gens_dict[tuple(g)]  if tuple(g) in gens_dict  else(float("-inf") if minf == 0 else float("inf")) for g in gens]
            alter_gens=[i for i,g in enumerate(gens) if tuple(g) not in gens_dict]
            #print(len(alter_gens))
            with Pool(mp) as p:
                alter_fit = p.map(estimate,[gens[i] for i in alter_gens])
            # pbest=dc(fit)
            # xpbest=dc(gens)
            # for i,f in enumerate(alter_fit):
            #     fnew[i]=f
            #     gens_dict[tuple(gens[i])]=f
            z=0
            for i in range(len(fit)):
                if i in alter_gens:
                    fnew[i]=alter_fit[z]
                    gens_dict[tuple(gens[i])]=alter_fit[z]
                    z+=1
                else:pass

                if fnew[i] >= fit[i] and random.random() < loud_A if minf==0 else fnew[i] <= fit[i] and random.random() < loud_A:#max?
                    gens[i]=gens[i]
                    fit[i]=fnew[i]

                if fnew[i]>best_v if minf==0 else fnew[i]<best_v:
                    best_s=dc(gens[i])
                    best_v=dc(fnew[i])
                    #print(gens_dict)
    print(len(gens_dict))
    return best_v,best_s,best_s.count(1)

In [4]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
 
from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import numpy as np

from sklearn import svm
 
SEED = 2018
np.random.seed(SEED)
df_train = pd.read_csv("VoxForge512.csv")
X = np.array(df_train.drop(['label'],axis=1))
y = np.array(df_train['label'])
from sklearn.model_selection import train_test_split
df=df_train
(a,b)=np.shape(df)
print(a,b)
data = df.values[:,0:b-1]
label = df.values[:,b-1]
# trainX=data
# trainy=label
cross=4
test_size=(1/cross)
trainX, testX, trainy, testy = train_test_split(data, label,stratify=label ,test_size=test_size)

900 201


In [5]:
if __name__ == '__main__':
    from sklearn import svm
    from time import time

    np.random.seed(20)
    tr_d=trainX
    te_d=testX
    tr_l=trainy
    te_l=testy

In [6]:
class Evaluate:#setting class
        def __init__(self):#set train_data,label,test_data,label
            self.train_l=tr_l
            self.train_d=tr_d
            self.test_l=te_l
            self.test_d=te_d
        def evaluate(self,gen):
            """
            Setting of evaluation function.
            Here, the correct answer rate is used.
              anser_label/all_label
            """
            mask=np.array(gen) > 0
            al_data=np.array([al[mask] for al in self.train_d])
            al_test_data=np.array([al[mask] for al in self.test_d])
            #↑masking with [01]sequence list
            res=RandomForestClassifier(n_jobs = -1, verbose = 0, n_estimators=5, criterion='entropy', random_state = 0).fit(al_data,self.train_l).predict(al_test_data)
            return np.count_nonzero(self.test_l==res)/len(self.test_l)
            #↑evaluate with fittness function
        def check_dimentions(self,dim):#check number of all feature
            if dim==None:
                return len(self.train_d[0])
            else:
                return dim

In [11]:
%%time
s,g,l=BBA(Eval_Func=Evaluate, n=5, m_i=5)

29
CPU times: user 1.15 s, sys: 89.4 ms, total: 1.24 s
Wall time: 6.22 s


In [12]:
l

72

In [13]:
List = []

for i in range(0,len(g)):
    if g[i] == 1:
        List.append(i)
        
df_train = pd.read_csv("VoxForge512.csv")
y = (df_train['label'])
df_train1 = df_train[df_train.columns[List]]
l1 = []
for i in range(0,len(List)):
    l1.append(i)
    
df_train1.columns = l1

df_train1['label'] = y
df_train1

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,label
0,0.002662,0.003032,0.002435,0.004380,0.002665,0.002319,0.000742,0.001880,0.000436,0.000497,...,0.288751,0.273255,0.426246,27.689117,30.487509,6.344035,6.411717,11.462402,0.016049,English
1,0.002068,0.001964,0.003271,0.001584,0.002996,0.001032,0.000193,0.001627,0.000170,0.000250,...,0.138344,0.131241,0.209426,14.013781,8.546353,13.334414,4.699897,11.421647,0.014899,English
2,0.001315,0.001354,0.000923,0.001206,0.001195,0.001047,0.000294,0.000597,0.000363,0.000330,...,0.418933,0.402468,0.715689,27.272018,24.049574,9.672536,6.692659,11.088083,0.014856,English
3,0.007111,0.011722,0.007152,0.008713,0.008530,0.003775,0.002010,0.002740,0.001000,0.001264,...,0.382544,0.397943,1.145464,34.190670,60.963062,6.542598,0.146815,9.626831,0.011871,English
4,0.002512,0.004911,0.004053,0.004527,0.003138,0.001852,0.001037,0.001629,0.000500,0.000424,...,0.485003,0.456029,0.668336,17.364416,40.051922,2.188263,2.625072,9.285885,0.011261,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,0.000608,0.001097,0.000490,0.000559,0.000726,0.000330,0.000327,0.000685,0.000301,0.000355,...,0.001296,0.001221,0.001838,0.006650,0.031571,0.004427,0.009261,29.929813,0.032924,Spanish
896,0.000111,0.000120,0.000100,0.000161,0.000092,0.000101,0.000090,0.000084,0.000129,0.000126,...,0.000634,0.000622,0.001290,0.020273,0.019685,0.006292,0.006949,9.986907,0.005506,Spanish
897,0.000059,0.000099,0.000062,0.000040,0.000042,0.000049,0.000028,0.000052,0.000037,0.000036,...,0.000539,0.000514,0.000858,0.001238,0.005864,0.012630,0.001491,9.939436,0.006557,Spanish
898,0.000005,0.000007,0.000006,0.000005,0.000008,0.000007,0.000017,0.000011,0.000008,0.000010,...,0.000183,0.000166,0.000162,0.000782,0.003085,0.002412,0.002749,9.965343,0.004320,Spanish


In [14]:
df_train1.to_csv('VF_BBA.csv')